In [117]:
# add OSM features to DAPHNE & INHALE trials

In [59]:
import pandas as pd
import numpy as np
from os import path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import data_process as dp
import draw_map as dm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, plot_confusion_matrix
import matplotlib.pyplot as plt

In [107]:
#inhale_dir = "./data/inhale"
#inhale_path = path.join(inhale_dir, "LIX005_airspeck_personal_automatic.csv")
#full_data = pd.read_csv(inhale_path)

#inhale_dir = "./data/daphne"
inhale_dir = "../data/DAPHNE/airspeck_AP"
inhale_path = path.join(inhale_dir, "DAP114(1)_airspeck_personal_manual.csv")
full_data = pd.read_csv(inhale_path)

In [61]:
delhi_road = dp.load_highway('delhi_new')

In [62]:
delhi_road.shape

(159522, 5)

In [63]:
np.unique(delhi_road['highway'].astype(str), return_counts=True)

(array(['footway', 'motorway', 'primary_secondary', 'residential',
        'tertiary', 'unclassified', 'unknown'], dtype=object),
 array([  5691,   1313,   5558, 130773,   5026,   4263,   6898]))

In [64]:
delhi_land = dp.load_landuse('inhale')

In [65]:
delhi_land.shape

(375551, 4)

In [66]:
np.unique(delhi_land['landuse'].astype(str), return_counts=True)

(array(['building', 'outland', 'residential'], dtype=object),
 array([179320,  21861, 174370]))

In [108]:
daphne = dp.add_road_feature(delhi_road, inhale_path)

In [109]:
daphne = dp.add_land_feature(delhi_land, daphne)

In [110]:
daphne = dp.distance_euclidean(daphne)

In [111]:
daphne = dp.calculate_std(daphne, 'gps_dist', k=10)

In [112]:
np.unique(daphne['landuse'].astype(str), return_counts=True)

(array(['unknown'], dtype=object), array([2457]))

In [113]:
np.unique(daphne['highway'].astype(str), return_counts=True)

(array(['footway', 'primary_secondary', 'residential', 'tertiary',
        'unclassified', 'unknown'], dtype=object),
 array([   1,  210, 2140,    1,   83,   22]))

In [114]:
bins = ['bin0', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8',
       'bin9', 'bin10', 'bin11', 'bin12', 'bin13', 'bin14', 'bin15']
daphne[bins] = daphne[bins].div(daphne[bins].sum(axis=1), axis=0)
for b in bins:
    daphne[b].fillna(0, inplace=True)

In [115]:
daphne.columns

Index(['timestamp', 'pm2_5', 'temperature', 'humidity', 'bin0', 'bin1', 'bin2',
       'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10',
       'bin11', 'bin12', 'bin13', 'bin14', 'bin15', 'total', 'gpsLongitude',
       'gpsLatitude', 'gpsAltitude', 'gpsAccuracy', 'luxLevel', 'motion',
       'battery', 'timestamp.1', 'geometry', 'highway', 'distance',
       'close_road_idx', 'landuse', 'close_land_idx', 'gps_dist',
       'gps_dist_std'],
      dtype='object')

In [116]:
out_dir = "./data/daphne/processed_data"
out_path = path.join(out_dir, "DAP114.csv")
daphne.to_csv(out_path)